In [6]:
# imports
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import json
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
import xgboost as xgb
import random
from scipy.stats import kendalltau
from scipy.stats import pearsonr
from sklearn.metrics import ndcg_score
import scipy.stats

In [7]:
data = np.load('X-y.npz')
X = data['X']
y = data['y']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=17)

# SMOTE to balance my dataset
smote = SMOTE(random_state=42)
x_sm, y_sm = smote.fit_resample(X_train, y_train)

X_train = np.concatenate((X_train, x_sm), axis = 0)
y_train = np.concatenate((y_train, y_sm), axis = 0)

In [8]:
def labels_to_integers(labels):
    indexed_labels = list(enumerate(labels))
    
    # Shuffle the labels to break ties randomly
    random.shuffle(indexed_labels)
    
    sorted_indexed_labels = sorted(indexed_labels, key=lambda x: x[1])
    index_to_int = {index: i+1 for i, (index, label) in enumerate(sorted_indexed_labels)}
    integer_labels = [index_to_int[i] for i in range(len(labels))]
    
    return integer_labels

X = np.array(X_train)
y = np.array(y_train)
qid = [1] * len(X_train)
ranker = xgb.XGBRanker(tree_method="hist", lambdarank_num_pair_per_sample=8, objective="rank:pairwise", lambdarank_pair_method="topk")
ranker.fit(X, y, qid=qid)

XGBRanker(base_score=None, booster=None, callbacks=None, colsample_bylevel=None,
          colsample_bynode=None, colsample_bytree=None, device=None,
          early_stopping_rounds=None, enable_categorical=False,
          eval_metric=None, feature_types=None, gamma=None, grow_policy=None,
          importance_type=None, interaction_constraints=None,
          lambdarank_num_pair_per_sample=8, lambdarank_pair_method='topk',
          learning_rate=None, max_bin=None, max_cat_threshold=None,
          max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
          max_leaves=None, min_child_weight=None, missing=nan,
          monotone_constraints=None, multi_strategy=None, n_estimators=None,
          n_jobs=None, ...)

In [9]:
scores = ranker.predict(X)
scores_and_values = zip(y_train, scores)
sorted_list = sorted(scores_and_values, key=lambda x: x[1], reverse=True)

test_embeddings = np.array(X_val)
pred_scores = ranker.predict(test_embeddings)
true_scores = np.array(y_val)

pred_scores_sorted_indices = np.argsort(pred_scores)
true_scores_sorted_indices = np.argsort(true_scores)

# Assign descending scores to the reference list
true_scores_sorted_indices = list(true_scores_sorted_indices)
pred_scores_sorted_indices = list(pred_scores_sorted_indices)
true_relevance = np.asarray([[len(true_scores_sorted_indices) - true_scores_sorted_indices.index(item) for item in true_scores_sorted_indices]])
# Assign scores to the second list based on their position in the reference list
scores = np.asarray([[len(true_scores_sorted_indices) - true_scores_sorted_indices.index(item) if item in true_scores_sorted_indices else 0 for item in pred_scores_sorted_indices]])

tau, p_value = kendalltau(pred_scores_sorted_indices, true_scores_sorted_indices)
print("xgboost ranker kendall tau: ", tau, "xgboost ranker p_value: ", p_value)
ranks1 = scipy.stats.rankdata(pred_scores_sorted_indices)
ranks2 = scipy.stats.rankdata(true_scores_sorted_indices)
pearson_corr, _ = pearsonr(ranks1, ranks2)
print("Xgboost ranker pearson correlation: ", pearson_corr)
# Calculate nDCG
ndcg = ndcg_score(true_relevance, scores)
ndcg_k10 = ndcg_score(true_relevance, scores, k = 10)
rev_ndcg_k10 = ndcg_score((1 / true_relevance), (1 / scores), k = 10)
print("xgboost ranker ndcg: ", ndcg)
print("ndcg @ k = 10: ", ndcg_k10)
print("reverse ndcg @ k = 10: ", rev_ndcg_k10)

xgboost ranker kendall tau:  -0.01572617946345976 xgboost ranker p_value:  0.8761136515296206
Xgboost ranker pearson correlation:  -0.015841813135985195
xgboost ranker ndcg:  0.8557834220250038
ndcg @ k = 10:  0.6269864744374263
reverse ndcg @ k = 10:  0.19266614641103988
